In [4]:
#Hacemos lectura del Dataset, el cual se llama: fraud_detection

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff

In [ ]:
fraude=pd.read_csv("C:/Users/jforero/OneDrive - Grupo Bancolombia/fraud_detection.csv")
fraude.head(10)

Descripción de las variables:

• step: Unidad de tiempo (1 step equivale a 1 hora).

• type: Tipo de transacción en línea.

• amount: Monto de la transacción.

• nameOrig: Cliente que inicia la transacción.

• oldbalanceOrg: Saldo del cliente antes de la transacción.

• newbalanceOrig: Saldo del cliente después de la transacción.

• nameDest: Destinatario de la transacción.

• oldbalanceDest: Saldo inicial del destinatario antes de la transacción.

• newbalanceDest: Nuevo saldo del destinatario después de la transacción.

• isFraud: Indicador de transacción fraudulenta (marcado como 1 si es fraudulenta).

• isFlaggedFraud: Indicador de transacción marcada por un sistema de alerta.

In [6]:
#Conteo de registros totales  columnas

print("Forma de la base de fraude (filas, columnas):", fraude.shape)
print(f"Total de registros: {fraude.shape[0]}")
print(f"Total de columnas: {fraude.shape[1]}")

Forma de la base de fraude (filas, columnas): (6362620, 11)
Total de registros: 6362620
Total de columnas: 11


In [ ]:
#Seguir conocimiento la base de datos y su tipo de dato, esto es importante si más adelante necesitamos realizar algunas operaciones.
fraude.info()

In [ ]:
#Identificando el tipo de dato de cada una de las coloumnas, se puede evidenciar que la coloumna type puede ser clave para el ejercicio, pero es una variable tipo object, la vamos a convertir en tipo category y así lograr un mayo detalle en los analisis
fraude['type'] = fraude['type'].astype('category')
print(fraude['type'].dtypes)

In [ ]:
#volvemos a correr la información del dataframe
fraude.info()

In [ ]:
# Tasa de fraude por tipo de transacción
fraude.groupby('type')['isFraud'].mean().sort_values(ascending=False)

In [ ]:
# Distribución de transacciones por tipo
fraude['type'].value_counts()

In [ ]:
#Concer las estadisticas a nivel general sin ninguna modificación del DataFrame.
fraude.describe()

In [ ]:
#Conocer número total de registros duplicados (todas las columnas iguales), esto me ayudará a evidenciar si existen registros duplicados y me pueden hacer ruido en el modelo.

duplicados = fraude.duplicated().sum()
print(f"Registros completamente duplicados: {duplicados}")

In [ ]:
#Ahora quiero identificar aquellos valores únicos por columna de algunos campos claves
valores_unicos = fraude.nunique()
print("Valores únicos por columna:")
print(valores_unicos)

In [ ]:
#Ahora, quiero identificar valores nulos por columna, y así identificar problemas de completitud

nulos = fraude.isnull().sum()
#nulos = nulos[nulos > 0]  # Mostrar solo columnas con al menos un nulo

print("Valores nulos por columna:")
print(nulos)

In [ ]:
#La columna clave es isFraud quiero hacer un conteo por cada una de las clases

print(fraude['isFraud'].value_counts())

Con el resultado anterior, se puede evidenciar que se tiene un problema de desbalanceo en la muestra, debido a que la cantidad de fraudes es solo un 0.13% del total de registros. Para esto existen técnicas de desbalanceo.

Procesamiento: A pesar que en el analisis inicial se hicieron algunas revisiones de posibles inconsistencias, en este capitulo se revisara la data a un mayor detalle, a continuación algunas validaciones

In [ ]:
#Revisare los nulos por cada una de las columnas de la base


fraude.isnull().sum()

Según la revisión anterior como no existen nulos no se hace necesario hacer una revisión de posibles opciones de nulos, las cuales podrían ser:

1. Eliminar los nulos si es un porcentaje pequeño (inferior al 1%)
2. Si tenemos muchos nulos tendríamos que buscar la manera de imputarlos, por ejemplo: a partir de media, mediana, entre otros

In [ ]:
#Ahora vamos a revisar datos inconsistentes o fuera del rango

#Validacion del monto de la transaccion negativo: 

fraude[fraude['amount'] < 0]

In [ ]:
#Verificación de valores negativos en saldos (que podrían ser inconsistentes)

saldo_vars = ['oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
fraude[saldo_vars].describe()

In [ ]:
#Filtar valores negativos

fraude[fraude[saldo_vars].lt(0).any(axis=1)]

In [21]:
#También es importante revisar la coherencia de los saldos

# Inconsistencias en origen
fraude['inconsistencia_origen'] = ((fraude['oldbalanceOrg'] - fraude['amount'] != fraude['newbalanceOrig']) & (fraude['isFraud'] == 1))

# Inconsistencias en destino
fraude['inconsistencia_destino'] = ((fraude['oldbalanceDest'] + fraude['amount'] != fraude['newbalanceDest']) & (fraude['isFraud'] == 1))

In [ ]:
# Validaciones de la variable namedest, si identificamos algún caso de cliente donde se repita la acción de fraude

conteo_destinatario = fraude['nameDest'].value_counts().head(50).index
fraude[fraude['nameDest'].isin(conteo_destinatario)].groupby(['nameDest', 'isFraud']).size().unstack(fill_value=0)

In [23]:
#También podríamos crear variables a partir de los salgos y así poder robustecer el dataframe para la realización del modelo

fraude['saldo_cambiado_origen'] = fraude['oldbalanceOrg'] - fraude['newbalanceOrig']
fraude['porcentaje_transferido'] = fraude['amount'] / (fraude['oldbalanceOrg'] + 1)  # +1 para evitar división por cero

In [ ]:
#Validamos de nuevo nuestra base

fraude.head(10)

In [ ]:
# después de la revisión inicial, el analisis grafico (hisotograma, corrleaciones, etc) y la revisión detallada de las varaibles, se crea el nuevo dataframe solo con las collumnas que vamos a utilizar para el modelo final:

variables_modelo = ['step','type',  'amount', 'oldbalanceOrg', 'newbalanceOrig', 'isFraud', 'oldbalanceDest', 'newbalanceDest', 'isFlaggedFraud', 'saldo_cambiado_origen', 'porcentaje_transferido']

# Crear el nuevo dataframe
fraude_final = fraude[variables_modelo].copy()

# Mostrar primeras filas para validar
fraude_final.head()

In [ ]:
#Tenemos una variable que es categoría llamada type, la idea es convertirla a Dummy para correr el modelo

fraude_final = pd.get_dummies(fraude_final, columns=['type'], drop_first=True)
fraude_final.info()
fraude_final.head()

In [ ]:
#Antes de correr el modelo, es valido revisar como quedaron las variabels, nos ayudamos a partir de graficas:
#Histograma de variables claves

fraude_final[['amount', 'oldbalanceOrg', 'saldo_cambiado_origen']].hist(bins=50, figsize=(12, 6))
plt.suptitle("Distribución de variables monetarias")
plt.show()

In [ ]:
numericas = fraude_final.select_dtypes(include='number')

plt.figure(figsize=(12, 8))
sns.heatmap(numericas.corr(), annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Matriz de correlación")
plt.show()